In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from commit_transformer.tokenizer import build_vocab
from commit_transformer.preprosser import CommitDataset
from commit_transformer.model import CombinedModel
import torch
from torch.utils.data import DataLoader

df = pd.read_csv(r'../datasets/dataset.csv', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df

train, _ = train_test_split(df, train_size=0.35, random_state=42)
train, test = train_test_split(train, test_size=0.3, random_state=42)
test, val = train_test_split(test, train_size=0.5, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

/tmp/ipykernel_1695/2062604134.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


In [2]:
# df = df[['message','diff','label']] 
# df

In [3]:
train_data = []
for index, row in train.iterrows():
  train_data.append([row['message'],row['diff'],row['label']])

val_data = []
for index, row in val.iterrows():
  val_data.append([row['message'],row['diff'],row['label']])
test_data = []
for index, row in test.iterrows():
  test_data.append([row['message'],row['diff'],row['label']])

In [4]:

# # Split the dataset
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# # Build vocabulary
vocab = build_vocab(train_data)

# Create Datasets and DataLoaders
train_dataset = CommitDataset(train_data, vocab)
val_dataset = CommitDataset(val_data, vocab)
test_dataset = CommitDataset(test_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [5]:
model = torch.load("transformer_entire_bert_model_900repo.pth")

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report

In [7]:
test_labels, test_predictions = model.evaluate(test_loader)
print(classification_report(test_labels,test_predictions,digits=4))

Validation Accuracy: 0.7140151515151515
Precision: 0.6193181818181818
Recall: 0.5647668393782384
F1-Score: 0.5907859078590786
              precision    recall  f1-score   support

           0     0.7614    0.8000    0.7802       335
           1     0.6193    0.5648    0.5908       193

    accuracy                         0.7140       528
   macro avg     0.6903    0.6824    0.6855       528
weighted avg     0.7094    0.7140    0.7110       528

